In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [3]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
using Test
using ForwardDiff
using SpecialFunctions
using Polynomials
using BenchmarkTools
using TransportMap
using QuadGK
using FastGaussQuadrature
using AdaptiveTransportMap: derivative, vander
using DiffResults
using QuadGK

┌ Info: Precompiling AdaptiveTransportMap [bdf749b0-1400-4207-80d3-e689c0e3f03d]
└ @ Base loading.jl:1260


### Basis

In [50]:
@btime quadgk(x->FamilyPhyHermite[5](x), 0, 1.0)

  2.069 μs (137 allocations: 2.34 KiB)


(0.5630021473036511, 1.4497070210950369e-11)

In [5]:
B = CstLinProHermite(5)
MB = MultiBasis(B, 10);

In [53]:
1.0 != 0.0

true

In [ ]:
MultiBasis()

In [19]:
@btime f = ExpandedFunction([MultiFunction(MB, rand(1:7,10)) for i=1:5], randn(5));

  1.561 μs (15 allocations: 1.28 KiB)


In [17]:
@btime f(randn(10))

  4.323 μs (248 allocations: 4.02 KiB)


0.00814365786685839

In [28]:
@btime ForwardDiff.gradient(f, randn(10))

  3.055 μs (50 allocations: 6.41 KiB)


10-element Array{Float64,1}:
  0.0
 -0.41481567821516313
  0.0
 -0.2054768734405355
 -0.1616353245698944
  0.06113183193954419
  0.06236459956882944
  0.02235240578579206
  0.16626206776483202
  0.3377408008797908

In [38]:
function timing() 
    f = MultiFunction(MB, rand(1:6,10));
    result = DiffResults.HessianResult(randn(10));
    result = ForwardDiff.hessian!(result, f, randn(10));
end

timing (generic function with 1 method)

In [42]:
function timing2() 
    f = MultiFunction(MB, rand(1:6,10));
    x = randn(10)
    f(x)
    ForwardDiff.gradient(f, x);
    ForwardDiff.hessian(f, x);
end

timing2 (generic function with 1 method)

In [43]:
@btime timing()

  6.722 μs (47 allocations: 51.77 KiB)


MutableDiffResult(-0.2044578068550304, ([0.39829120904469095, -0.19262095275384977, -0.06467058375300636, -0.16597678410144642, 0.09076585820500507, -0.4505810392251209, -0.10561444535867072, 0.12783878526911294, -0.11042588440504841, 0.10070902078475591], [0.627576166543466 0.3752325887661919 0.12598063820020433 0.32332878372302787 -0.17681517747122896 0.8777481752645109 0.20574076275950454 -0.24903458141728996 0.21511362019402344 -0.1961848181151047; 0.3752325887661919 0.0 -0.060926553254502115 -0.15636774540631362 0.08551107122735313 -0.4244951582108158 -0.09950001617683905 0.12043770299703523 -0.1040329023868896 0.09487858562535327; 0.12598063820020433 -0.060926553254502115 0.08177341450641945 -0.052498927198695755 0.02870949818571752 -0.14252005967855214 -0.0334061483841412 0.04043577008282555 -0.03492802019039548 0.03185454868916674; 0.32332878372302787 -0.15636774540631362 -0.052498927198695755 0.5065202209408731 0.07368280763060513 -0.36577714012505186 -0.08573674082163721 0.10

In [44]:
@btime timing2()

  12.227 μs (74 allocations: 46.17 KiB)


10×10 Array{Float64,2}:
  0.0        0.121232    0.0   0.656108   …  0.0  -0.856951   0.0   1.31124
  0.121232  -0.505286    0.0   0.0285255     0.0  -0.0372576  0.0   0.0570086
  0.0        0.0         0.0   0.0           0.0   0.0        0.0   0.0
  0.656108   0.0285255   0.0  -0.487653      0.0  -0.201639   0.0   0.308532
  0.21697    0.00943319  0.0   0.0510526     0.0  -0.0666804  0.0   0.102029
 -0.12933   -0.00562285  0.0  -0.0304309  …  0.0   0.0397462  0.0  -0.0608166
  0.0        0.0         0.0   0.0           0.0   0.0        0.0   0.0
 -0.856951  -0.0372576   0.0  -0.201639      0.0  -0.376055   0.0  -0.402977
  0.0        0.0         0.0   0.0           0.0   0.0        0.0   0.0
  1.31124    0.0570086   0.0   0.308532      0.0  -0.402977   0.0   0.0

### Hermite functions

In [27]:
@time integral, err = quadgk(x -> FamilyPhyHermite[5](x), 0.0, 1.0, rtol=1e-8)

  0.069184 seconds (167.45 k allocations: 8.917 MiB)


(0.5630021473036511, 1.4497070210950369e-11)

In [29]:
derivative(FamilyPhyHermite[5],-1, [1.0])

1-element Array{Float64,1}:
 -18.84597896350062